## **<font color="red">Introduction to Conversational Context: Session, State, and Memory</font>**
- Meaningful, multi-turn conversations require agents to understand context. Just like humans, they need to recall the conversation history: what's been said and done to maintain continuity and avoid repetition. The **Agent Development Kit (ADK)** provides structured ways to manage this context through `Session`, `State`, and `Memory`.
### **Core Concepts**
- Think of different instances of your conversations with the agent as distinct **conversation threads**, potentially drawing upon **long-term knowledge**.
  1. `Session`: The Current Conversation Thread
     - Represents a _single_, ongoing _interaction_ between a user and your agent system.
     - Contains the chronological sequence of messages and actions taken by the agent (referred to `Events`) during that specific interaction.
     - A `Session` can also hold temporary data (`State`) relevant only during this conversation.
  2. `State` (`session.state`): Data within the Current Conversation
     - Data stored within a specific `Session`.
     - Used to manage information relevant only to the current, active conversation thread (e.g., items in a shopping cart during this chat, user preferences mentioned in this session).
  3. `Memory`: Searchable, Cross-Session Information
     - Represents a store of information that might span multiple past sessions or include external data sources.
     - It acts as a knowledge base the agent can search to recall information or context beyond the immediate conversation.
### **Managing Context: Services**
ADK provides services to manage these concepts:
1. `SessionService`: Manages the different conversation threads (`Session` objects)
   - Handles the lifecycle: creating, retrieving, updating (appending `Events`, modifying `State`), and deleting individual `Session`.
2. `MemoryService`: Manages the Long-Term Knowledge Store (`Memory`)
   - Handles ingesting information (often from completed `Session`) into the long-term store.
   - Provides methods to search this stored knowledge based on queries.
- **Implementations:** ADK offers different implementations for both `SessionService` and `MemoryService`, allowing you to choose the storage backend that best fits your application's needs. Notably, **in-memory implementations** are provided for both services; these are designed specifically for **local testing and fast development**. It's important to remember that **all data stored using these in-memory options (sessions, state, or long-term knowledge) is lost when your application restarts**. For persistence and scalability beyond local testing, ADK also offers cloud-based and database service options.

## **<font color="red">Session: Tracking Individual Conversations</font>**
### **The `Session` Object**
- When a user starts interacting with your agent, the `SessionService` creates a `Session` object (`google.adk.sessions.Session`). This object acts as the container holding everything related to that one specific chat thread. Here are its key properties:
  - **Identification (`id`, `appName`, `userId`):** Unique labels for the conversation.
  - `id`: A unique identifier for this specific conversation thread, essential for retrieving it later. A SessionService object can handle multiple `Session`. This field identifies which aprticualr session object are we referring to. For example, "test_id_modification".
  - `app_name`: Identifies which agent application this conversation belongs to. For example, "id_modifier_workflow".
  - `userId`: Links the conversation to a particualr user.
- **History (`events`):** A chronological sequence of all interactions (Event objects – user messages, agent responses, tool actions) that have occurred within this specific thread.
- **Session State (`state`):** A place to store temporary data relevant only to this specific, ongoing conversation. This acts as a scratchpad for the agent during the interaction. We will cover how to use and manage `state` in detail in the next section.
- **Activity Tracking (`lastUpdateTime`):** A timeestamp indicating the last time an event occurred in this conversation thread.


In [1]:
 from google.adk.sessions import InMemorySessionService, Session

 # Create a simple session to examine its properties
 temp_service = InMemorySessionService()
 example_session = await temp_service.create_session(
     app_name="my_app",
     user_id="example_user",
     state={"initial_key": "initial_value"} # State can be initialized
 )

 print(f"--- Examining Session Properties ---")
 print(f"ID (`id`):                {example_session.id}")
 print(f"Application Name (`app_name`): {example_session.app_name}")
 print(f"User ID (`user_id`):         {example_session.user_id}")
 print(f"State (`state`):           {example_session.state}") # Note: Only shows initial state here
 print(f"Events (`events`):         {example_session.events}") # Initially empty
 print(f"Last Update (`last_update_time`): {example_session.last_update_time:.2f}")
 print(f"---------------------------------")

 # Clean up (optional for this example)
 temp_service = await temp_service.delete_session(app_name=example_session.app_name,
                             user_id=example_session.user_id, session_id=example_session.id)
 print("The final status of temp_service - ", temp_service)


--- Examining Session Properties ---
ID (`id`):                bb4f1591-9dea-42b0-a4b9-e11fbc0389af
Application Name (`app_name`): my_app
User ID (`user_id`):         example_user
State (`state`):           {'initial_key': 'initial_value'}
Events (`events`):         []
Last Update (`last_update_time`): 1771178557.74
---------------------------------
The final status of temp_service -  None


### **Managing Sessions with a SessionService**
- As seen above, you don't typically create or manage Session objects directly. Instead, you use a SessionService. This service acts as the central manager responsible for the entire lifecycle of your conversation sessions.
- Its core responsibilities include:
  - **Starting New Conversations:** Creating fresh Session objects when a user begins an interaction.
  - **Resuming Existing Conversations:** Retrieving a specific Session (using its ID) so the agent can continue where it left off.
  - **Saving Progress:** Appending new interactions (Event objects) to a session's history. This is also the mechanism through which session state gets updated (more in the State section).
  - **Listing Conversations:** Finding the active session threads for a particular user and application.
  - **Cleaning Up:** Deleting Session objects and their associated data when conversations are finished or no longer needed.

### **`SessionService` Implementations**
- ADK provides different `SessionService` implementations, allowing you to choose the storage backend that best suits your needs:
- `InMemorySessionService`
  - **How it works:** Stores all session data directly in the application's memory.
  - **Persistence:** None. All conversation data is lost if the application restarts.
  - **Requires:** Nothing extra.
  - **Best for:** Quick development, local testing, examples, and scenarios where long-term persistence isn't required.
### **`VertexAiSessionService`**
- **How it works:** Uses Google Cloud Vertex AI infrastructure via API calls for session management.
- **Persistence:** Yes. Data is managed reliably and scalably via Vertex AI Agent Engine.
- **Requires:**
  - A Google Cloud project (pip install vertexai)
  - A Google Cloud storage bucket that can be configured by this step.
  - A Reasoning Engine resource name/ID that can setup following this tutorial.
  - If you do not have a Google Cloud project and you want to try the VertexAiSessionService, see Vertex AI Express Mode.
- **Best for:** Scalable production applications deployed on Google Cloud, especially when integrating with other Vertex AI features.

In [2]:
# Requires: pip install google-adk[vertexai]
# Plus GCP setup and authentication
from google.adk.sessions import VertexAiSessionService

PROJECT_ID = "your-gcp-project-id"
LOCATION = "us-central1"
# The app_name used with this service should be the Reasoning Engine ID or name
REASONING_ENGINE_APP_NAME = "projects/your-gcp-project-id/locations/us-central1/reasoningEngines/your-engine-id"

session_service = VertexAiSessionService(project=PROJECT_ID, location=LOCATION)
# Use REASONING_ENGINE_APP_NAME when calling service methods, e.g.:
# session_service = await session_service.create_session(app_name=REASONING_ENGINE_APP_NAME, ...)

### **`DatabaseSessionService`**
- **How it works:** Connects to a relational database (e.g., _PostgreSQL_, _MySQL_, _SQLite_) to store session data persistently in tables.
- **Persistence:** Yes. Data survives application restarts.
- **Requires:** A configured database.
- **Best for:** Applications needing reliable, persistent storage that you manage yourself.

In [3]:
from google.adk.sessions import DatabaseSessionService
# Example using a local SQLite file:
# Note: The implementation requires an async database driver.
# For SQLite, use 'sqlite+aiosqlite' instead of 'sqlite' to ensure async compatibility.
db_url = "sqlite+aiosqlite:///./my_agent_data.db"
session_service = DatabaseSessionService(db_url=db_url)